 # Importar arquivos para leitura

In [1]:
import pandas as pd
import os

In [2]:
base_sip_oem = pd.read_excel(r"E:\OEM\content\base_sip_Concluido.xlsx")
#df_intelbras = pd.read_excel(r"E:\ACRÉSCIMOS PREVENTIVA JUNHO 3ª REMESSA.xlsx")


In [3]:
base_sip_oem = base_sip_oem[['IDSIGFI','NUMOS','NOMECLIENTE','CPF','CODIGOENERGISA','UC','TIPODESIGFI','OBRA','ROTA','EXECUTOR','LATLONCON','TIPOCAUSA','DTCONCLUSAO','ENDERECO','MUNICIPIO']]


In [4]:
print(base_sip_oem.shape)
#print(df_intelbras.shape)

(8577, 15)


# Realizar leitura de arquivos  .pdf  dentro das pastas correspondente

In [5]:
diretorio_lote = r'C:\Users\helym\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\00-INTELBRAS\000_Relatórios de Campo\001 RELATORIOS ENVIADOS\AC\0_0-PREVENTIVA\1ªTRACHE\VISITA TÉCNICA'
#"C:\Users\OneEngInst\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\00-INTELBRAS\000_Relatórios de Campo\001 RELATORIOS ENVIADOS\0.0-PREVENTIVA\1ªTRACHE\2024\LOTE_16\3ªREMESSA"

In [6]:
# listando os arquivos dentro da pasta.
arquivos_pdf = []
for raiz, diretorios, arquivos in os.walk(diretorio_lote):
    for arquivo in arquivos:
        if arquivo.endswith('.pdf'):
            arquivos_pdf.append(os.path.join(raiz, arquivo))

lista_pdf = pd.DataFrame(arquivos_pdf, columns=['NUMOS'])
lista_pdf['NUMOS'] = lista_pdf['NUMOS'].apply(lambda x: x[-22:-10]).astype('int64')
contagem_arquivos = lista_pdf['NUMOS'].unique().shape[0]
dataset = lista_pdf.merge(base_sip_oem, left_on='NUMOS', right_on='NUMOS', how='left')

In [7]:
dataset[['LATLONCON']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   LATLONCON  9 non-null      object
dtypes: object(1)
memory usage: 204.0+ bytes


In [8]:
# Rastrear duplicidade e soma-las.
dataset['NUMOS'].duplicated().sum()

0

In [9]:
dataset['NUMOS'].count()

9

In [10]:
def encontrar_duplicados(dataset):
    duplicados = dataset[dataset.duplicated(subset='NUMOS')]
    return duplicados['NUMOS']

# Exemplo de uso
duplicados = encontrar_duplicados(dataset)
print(duplicados)


Series([], Name: NUMOS, dtype: int64)


# Realizar a mesclagem com número de OS entre os dados da base do sistema com a lista de arquivos encontrados na pasta

In [11]:
#dataset[['LATITUDE', 'LONGITUDE','ERRO_1','ERRO_2']] = dataset['LATLONCON'].apply(lambda x: pd.Series(x.split(',')))#,'ERRO_1','ERRO_2'
def split_latlon(x):
    if isinstance(x, str):
        return pd.Series(x.split(','))
    else:
        #Se não for uma string, retorna NaN ou valores padrão, dependendo do que você deseja
        return pd.Series([pd.NaT, pd.NaT])

# Aplica a função split_latlon à coluna 'LATLONCON'
dataset[['LATITUDE', 'LONGITUDE']] = dataset['LATLONCON'].apply(split_latlon)#,'erro_1','erro_2'

# Converta as colunas 'LATITUDE' e 'LONGITUDE' para números, se necessário
#dataset[['LATITUDE', 'LONGITUDE']] = dataset[['LATITUDE', 'LONGITUDE']].apply(pd.to_numeric, errors='coerce')

dataset= dataset[['IDSIGFI','UC','OBRA','NOMECLIENTE','CPF','ENDERECO','MUNICIPIO','DTCONCLUSAO','TIPOCAUSA','ROTA','EXECUTOR','TIPODESIGFI','LATITUDE','LONGITUDE','NUMOS']]#,'NUMOS'
lote = dataset.rename(columns={'IDSIGFI':'CÓDIGO ONE','NOMECLIENTE':'NOME DO CLIENTE','CODIGOENERGISA': 'CÓDIGO','UC': 'NÚM.DA UC',
                               'OBRA':'NÚM.DA OBRA','EXECUTOR':'EQUIPE EXECUTORA',
                              'TIPODESIGFI': 'TIPO/SIGFI','TIPOCAUSA':'PREVENTIVA','DTCONCLUSAO':'DATA ATENDIMENTO'})

replacements1 = {'ONEOM001':'EQ.1','ONEOM002':'EQ.2','ONEOM003':'EQ.3','ONEOM004':'EQ.4','ONEOM005':'EQ.5','ONEOM006':'EQ.6','ONEOM007':'EQ.7',
                 'ONEOM008':'EQ.8','ONEOM009':'EQ.9','ONEOM010':'EQ.10','ONEOM011':'EQ.11','ONEOM012':'EQ.12','ONEOM013':'EQ.13','ONEOM014':'EQ.14',
                 'ONEOM015':'EQ.15'}
lote['EQUIPE EXECUTORA'] = lote['EQUIPE EXECUTORA'].replace(replacements1)

replacements2 = {'MANUT PREVENTIVA 1A':'1ªPREVENTIVA','MANUT PREVENTIVA 2A':'2ªPREVENTIVA','MANUT PREVENTIVA 3A':'3ªPREVENTIVA',
                 'MANUT PREVENTIVA 4A':'4ªPREVENTIVA','MANUT PREVENTIVA 5A':'5ªPREVENTIVA'}
lote['PREVENTIVA'] = lote['PREVENTIVA'].replace(replacements2)

replacements3 = {'CRUZEIRO DO SUL':'CRUZEIRO DO SUL','FEIJO':'CRUZEIRO DO SUL','JORDAO':'CRUZEIRO DO SUL',
                 'JURUA':'CRUZEIRO DO SUL','MARECHAL THAUMATURGO':'CRUZEIRO DO SUL','MANCIO LIMA':'CRUZEIRO DO SUL',
                'PORTO WALTER':'CRUZEIRO DO SUL','RODRIGUES ALVES':'CRUZEIRO DO SUL','TARAUACA':'CRUZEIRO DO SUL',
                'ASSIS BRASIL':'BRASILÉIA','SENA MADUREIRA':'RIO BRANCO','RIO AZUL':'CRUZEIRO DO SUL','xxxxxxxxx':'xxxxxxxxxx'}

lote['REGIONAL'] = lote['MUNICIPIO'].replace(replacements3)

lote = lote[['CÓDIGO ONE', 'NÚM.DA UC', 'NÚM.DA OBRA', 'NOME DO CLIENTE', 'CPF',
       'ENDERECO', 'MUNICIPIO','REGIONAL', 'DATA ATENDIMENTO', 'PREVENTIVA', 'ROTA',
       'EQUIPE EXECUTORA', 'TIPO/SIGFI', 'LATITUDE', 'LONGITUDE', 'NUMOS']]


In [12]:
lote

,CÓDIGO ONE,NÚM.DA UC,NÚM.DA OBRA,NOME DO CLIENTE,CPF,ENDERECO,MUNICIPIO,REGIONAL,DATA ATENDIMENTO,PREVENTIVA,ROTA,EQUIPE EXECUTORA,TIPO/SIGFI,LATITUDE,LONGITUDE,NUMOS
0,13521,642482.0,212200503.0,RANIELE CASTRO DA SILVA,053.105.462-43,COMUNIDADE SANTA RITA S/N RIO LIBERDADE CENTRO...,CRUZEIRO DO SUL,CRUZEIRO DO SUL,2024-05-07 09:30:00,INTERMITENCIA,6.0,EQ.7,SIGFI80,-7.72470305,-71.99884247,202400030807
1,13492,651979.0,212200683.0,JOSE FRANCISCO REINALDO TEODORO,706.336.242-92,COMUNIDADE SANTA RITA S/N LAGO SAO JOAO CENTRO...,CRUZEIRO DO SUL,CRUZEIRO DO SUL,2024-05-07 15:06:00,MANUT PREVENTIVA 4A,6.0,EQ.7,SIGFI80,-7.68164934,-71.95136345,202400030799
2,13888,652183.0,212200996.0,CRISTIANA DA SILVA NASCIMENTO,994.096.902-34,COMUNIDADE PEDRA PINTADA S/N RIO JURUA CENTRO ...,MARECHAL THAUMATURGO,CRUZEIRO DO SUL,2024-05-06 14:33:00,INTERMITENCIA,3.0,EQ.14,SIGFI80,-9.37534,-72.71380500000001,202400030716
3,15091,654905.0,212201489.0,RONISON SOARES DE AZEVEDO,084.526.512-10,RIO JURUA COMUNIDADE ADÃO E EVA RIO JURUA C...,MARECHAL THAUMATURGO,CRUZEIRO DO SUL,2024-05-11 10:23:00,MANUT PREVENTIVA 4A,3.0,EQ.3,SIGFI80,-9.177375000000001,-72.69680166666667,202400032142
4,14943,654934.0,212201530.0,MANELIZE OLIVEIRA BANDEIRA,036.392.252-01,RIO JURUA COMUNIDADE LUCANIA RIO JURUA C...,CRUZEIRO DO SUL,CRUZEIRO DO SUL,2024-05-05 11:57:00,INTERMITENCIA,1.0,EQ.7,SIGFI80,-8.11959252,-72.78867842,202400030690
5,13477,658167.0,212201288.0,FRANCISCO FIRMINO,632.937.492-91,COMUNIDADE MUNDURUCUS S/N RIO JURUA CENTRO COM...,RODRIGUES ALVES,CRUZEIRO DO SUL,2024-05-06 10:30:00,INTERMITENCIA,1.0,EQ.7,SIGFI80,-7.87165408,-72.72349525,202400030682
6,15055,658244.0,212202047.0,ANTONIO EUDES SANTOS DA SILVA,994.390.002-44,RIO JURUA COMUNIDADE TAPAUNA RIO JURUA COMU...,MARECHAL THAUMATURGO,CRUZEIRO DO SUL,2024-05-05 16:12:00,INTERMITENCIA,3.0,EQ.14,SIGFI80,-9.12992,-72.70863666666668,202400030708
7,15613,660907.0,112203279.0,FRANCINETE SIQUEIRA DE MOURA,615.393.252-53,SERINGAL GRANJA SERINGAL GRANJA,SENA MADUREIRA,RIO BRANCO,2024-05-08 14:50:00,INTERMITENCIA,11.0,EQ.6,SIGFI45,-9.15476644,-69.10088214,202400030849
8,15799,688884.0,NaN,FRANCISCO DAS CHAGAS DE AMORIM SILVA,006.641.342-70,BR 364. RIO TAUARI. COLÔNIA BELA VISTA BR 3...,TARAUACA,CRUZEIRO DO SUL,2024-04-25 10:59:00,MANUT PREVENTIVA 3A,13.0,EQ.7,SIGFI45,-7.93257324,-71.62246386,202400020154


In [13]:
# Salvando a planilha( .xlsx)
dataframe_excel = pd.ExcelWriter(r"E:\OEM\output\1ºtrache_intebras_ac_preventiva_visita.xlsx")#r"E:\OEM\output\1ºtrache_intebras_ac_preventiva_lote17_3ªremessa_2024.xlsx
lote.to_excel(dataframe_excel, index=False)
dataframe_excel.close()